# CSC-786. Activity 4 - Manuel Marin
## Download this notebook and open it in Google Colab.


## Setup Environment

In [ ]:
!pip install python-dotenv --quiet
import os, pandas as pd, requests, hashlib, json, sys, time, matplotlib
from datetime import datetime, timezone
from pathlib import Path
import matplotlib.pyplot as plt


ROOT = Path("/content/csc786-a4-manuel-marin")
DATA = ROOT / "data"
DATA.mkdir(exist_ok=True)
print("Environment ready. Files will be stored in:", DATA)


# API Key Setup
## Get your API Key from https://fdc.nal.usda.gov/api-key-signup

In [ ]:
# Store key securely in this Colab session
%env FOOD_DATA_API_KEY=your_api_key_here

API_KEY = os.getenv("FOOD_DATA_API_KEY")
print("Key loaded:", API_KEY[:6] + "****" if API_KEY else "No key found")


### USDA FoodData Central API - Get list of foods with comprehensive data

In [ ]:

url = "https://api.nal.usda.gov/fdc/v1/foods/list"
params = {
    "api_key": API_KEY,
    "pageSize": 50,      # Get more foods for better plotting
    "pageNumber": 1
}

r = requests.get(url, params=params, timeout=10)
r.raise_for_status()
data = r.json()

# Extract comprehensive food data for plotting
foods_data = []
for food in data:
    food_info = {
        "fdcId": food.get("fdcId"),
        "description": food.get("description"),
        "dataType": food.get("dataType"),  # Good for categorical plotting
        "publicationDate": food.get("publicationDate", "N/A"),
        "brandOwner": food.get("brandOwner", "N/A"),
    }

    # Extract key nutrients for plotting
    nutrients = food.get("foodNutrients", [])
    nutrient_dict = {}

    for nutrient in nutrients:
        nutrient_name = nutrient.get("nutrientName", "")
        nutrient_value = nutrient.get("value", 0)
        nutrient_unit = nutrient.get("unitName", "")

        # Store common nutrients useful for plotting
        if "Energy" in nutrient_name or "Protein" in nutrient_name or \
            "Carbohydrate" in nutrient_name or "Fat" in nutrient_name or \
            "Fiber" in nutrient_name or "Sugar" in nutrient_name:
            nutrient_dict[nutrient_name] = {
                "value": nutrient_value,
                "unit": nutrient_unit
            }

    # Add nutrients to food_info
    food_info["nutrients"] = nutrient_dict
    food_info["nutrient_count"] = len(nutrients)  # Track how many nutrients available

    foods_data.append(food_info)

foods_data


In [ ]:
df = pd.DataFrame(foods_data)

print(f"Retrieved {len(df)} foods")
print(f"\nData types available: {df['dataType'].unique()}")
print(f"\nColumns available: {df.columns.tolist()}")
df.head()


# Step 1: Search for commonly wasted restaurant foods

In [ ]:
commonly_wasted = ["french fries", "bread", "pizza", "salad", "chicken wings"]
SEARCH_ENDPOINT = "https://api.nal.usda.gov/fdc/v1/foods/search"

all_foods = []

for food_name in commonly_wasted:
    params = {
        "api_key": API_KEY,
        "query": food_name,
        "pageSize": 1
    }

    try:
        r = requests.get(SEARCH_ENDPOINT, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()

        if data.get("foods"):
            food = data["foods"][0]

            nutrients_dict = {}
            for nutrient in food.get("foodNutrients", []):
                name = nutrient.get("nutrientName", "")
                value = nutrient.get("value", 0)

                if "Energy" in name:
                    nutrients_dict["Calories"] = value
                elif name == "Protein":
                    nutrients_dict["Protein (g)"] = value
                elif name == "Total lipid (fat)":
                    nutrients_dict["Fat (g)"] = value
                elif name == "Carbohydrate, by difference":
                    nutrients_dict["Carbs (g)"] = value

            all_foods.append({
                "food_name": food.get("description"),
                "search_term": food_name,
                "fdcId": food.get("fdcId"),
                "dataType": food.get("dataType"),
                **nutrients_dict
            })

            print(f"✓ Found: {food.get('description')}")

    except Exception as e:
        print(f"✗ Error fetching {food_name}: {e}")


## Step 2: Create DataFrame

In [ ]:
df = pd.DataFrame(all_foods)

if "Calories" not in df.columns:
    print("\n⚠ Calories not found. Calculating from macronutrients...")
    df["Calories"] = (
        df.get("Protein (g)", 0) * 4 +
        df.get("Carbs (g)", 0) * 4 +
        df.get("Fat (g)", 0) * 9
    )
    calories_calculated = True
    print("✓ Calories calculated")
else:
    calories_calculated = False

print(f"\nDataFrame created with {len(df)} foods")
print(df)


## Step 3: Generate timestamp ONCE for all files


In [ ]:
timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H%M%SZ")

## Step 4: Save CSV

In [ ]:
out_csv = DATA / f"food_nutrition_data_{timestamp}.csv"
df.to_csv(out_csv, index=False)
print(f"\n✓ Data saved to CSV: {out_csv.name}")

# Calculate hash
file_hash = hashlib.sha256(out_csv.read_bytes()).hexdigest()
print(f"✓ File hash: {file_hash[:16]}...")

## Step 5: Create Chart 1 - Calories

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
df_sorted = df.sort_values('Calories', ascending=True)
colors = ['#e74c3c', '#e67e22', '#f39c12', '#27ae60', '#3498db']
bars = ax.barh(df_sorted['search_term'], df_sorted['Calories'], color=colors)

ax.set_xlabel('Calories per 100g', fontsize=12, weight='bold')
ax.set_ylabel('Commonly Wasted Restaurant Foods', fontsize=12, weight='bold')
ax.set_title('Nutritional Value Lost: Calories in Commonly Wasted Foods',
              fontsize=14, weight='bold', pad=20)
ax.grid(axis='x', alpha=0.3, linestyle='--')

for bar, val in zip(bars, df_sorted['Calories']):
    ax.text(val + 5, bar.get_y() + bar.get_height()/2,
            f'{val:.0f} kcal', va='center', fontsize=10, weight='bold')

plt.tight_layout()
plot_file1 = DATA / f"food_waste_calories_{timestamp}.png"
plt.savefig(plot_file1, dpi=300, bbox_inches='tight')
print(f"\n✓ Plot 1 saved: {plot_file1.name}")
plt.show()


## Step 6: Create Chart 2 - Nutrients

In [ ]:
nutrient_cols = [col for col in df.columns if col not in ['food_name', 'search_term', 'Calories', 'fdcId', 'dataType']]

plot_file2 = None
if len(nutrient_cols) > 0:
    fig, ax = plt.subplots(figsize=(12, 6))

    x = range(len(df))
    width = 0.25
    colors_nutrients = ['#3498db', '#e74c3c', '#f39c12']

    for i, col in enumerate(nutrient_cols):
        if col in df.columns:
            offset = width * (i - len(nutrient_cols)/2)
            ax.bar([p + offset for p in x], df[col], width,
                    label=col, color=colors_nutrients[i % len(colors_nutrients)])

    ax.set_xlabel('Foods', fontsize=12, weight='bold')
    ax.set_ylabel('Amount (grams per 100g)', fontsize=12, weight='bold')
    ax.set_title('Macronutrient Comparison: Commonly Wasted Foods',
                  fontsize=14, weight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(df['search_term'], rotation=45, ha='right')
    ax.legend(loc='upper left')
    ax.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plot_file2 = DATA / f"food_waste_nutrients_{timestamp}.png"
    plt.savefig(plot_file2, dpi=300, bbox_inches='tight')
    print(f"✓ Plot 2 saved: {plot_file2.name}")
    plt.show()


## Step 7: LOG PROVENANCE TO DATA_README.md

In [ ]:
meta = {
    "timestamp_utc": timestamp,
    "endpoint": SEARCH_ENDPOINT,
    "search_terms": commonly_wasted,
    "foods_collected": len(df),
    "params": {
        "pageSize": 1,
        "api_key": "REDACTED"
    },
    "output_csv": out_csv.name,
    "output_plot1": plot_file1.name,
    "output_plot2": plot_file2.name if plot_file2 else None,
    "sha256_csv": file_hash,
    "calories_calculated": calories_calculated,
    "python": sys.version.split()[0],
    "pandas": pd.__version__,
    "requests": requests.__version__,
    "matplotlib": matplotlib.__version__,
}

with open(ROOT / "DATA_README.md", "a") as f:
    f.write(f"\n\n```json\n{json.dumps(meta, indent=2)}\n```")

print(f"\n✓ Metadata logged to DATA_README.md")

# Step 8: Display Summary

In [ ]:

print("\n" + "="*60)
print("📊 DATA COLLECTION SUMMARY")
print("="*60)
print(f"Timestamp:       {timestamp}")
print(f"Foods collected: {len(df)}")
print(f"CSV file:        {out_csv.name}")
print(f"Chart 1:         {plot_file1.name}")
if plot_file2:
    print(f"Chart 2:         {plot_file2.name}")
print(f"SHA-256 hash:    {file_hash[:16]}...")
print("="*60)

# Display last entries in DATA_README.md
print("\n📄 Latest DATA_README.md entries:")
!tail -n 25 {ROOT / "DATA_README.md"}

## **Calculate calories if missing**

In [ ]:
df = pd.DataFrame(all_foods)

if "Calories" not in df.columns:
    print("\n⚠ Calories not found in API response. Calculating from macronutrients...")
    # Calorie calculation: Protein(4 cal/g) + Carbs(4 cal/g) + Fat(9 cal/g)
    df["Calories"] = (
        df.get("Protein (g)", 0) * 4 +
        df.get("Carbs (g)", 0) * 4 +
        df.get("Fat (g)", 0) * 9
    )
    print("✓ Calories calculated successfully")

print(f"\nDataFrame created with {len(df)} foods")
print(df)

# ========================================
# NEW: Save DataFrame to CSV
# ========================================
timestamp = datetime.now(timezone.utc).strftime("%Y-%m-%dT%H%M%SZ")
csv_file = DATA / f"food_nutrition_data_{timestamp}.csv"
df.to_csv(csv_file, index=False)
print(f"\n✓ Data saved to CSV: {csv_file}")

# Calculate SHA-256 hash of CSV file for provenance
file_hash = hashlib.sha256(csv_file.read_bytes()).hexdigest()
print(f"✓ File hash (SHA-256): {file_hash[:16]}...")
